# IMDB XGBoost Total Score
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

# Importing so that I can use LinearRegression and OLS Models.
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics

In [2]:
totalscore_df = pd.read_csv('../data/totalscore_df.csv')

In [3]:
totalscore_df.columns

Index(['movie_title', 'year', 'actors', 'plot', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'votes',
       'weighted_average_vote', 'total_votes', 'mean_vote', 'median_vote',
       'votes_1', 'votes_2', 'votes_3', 'votes_4', 'votes_5', 'votes_6',
       'votes_7', 'votes_8', 'votes_9', 'votes_10', 'us_voters_rating',
       'us_voters_votes', 'plot_sentiment', 'director_score', 'actor_score',
       'actress_score', 'imdb_score'],
      dtype='object')

### XGBoost

In [4]:
from xgboost import XGBRegressor

In [5]:
xgb = XGBRegressor()

In [6]:
features = ['duration', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'votes',
       'votes_2',  'votes_6',
       'votes_7', 'plot_sentiment', 'director_score', 'actor_score',
       'actress_score']

In [7]:
X = totalscore_df[features]
y = totalscore_df['imdb_score']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [9]:
xgb.fit(X_train, y_train);

In [10]:
print("Training Score:", xgb.score(X_train, y_train))
print(" ")
print("Testing Score:", xgb.score(X_test, y_test))

Training Score: 0.998554503519591
 
Testing Score: 0.9429373281640024


In [11]:
predictions = xgb.predict(X_train)

residuals = y_train - predictions

rss = (residuals ** 2).sum()

mse = metrics.mean_squared_error(y_train, predictions)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))

In [12]:
null_predictions = np.zeros_like(y_test)

null_pred = null_predictions + y_test.mean()

baseline = metrics.mean_squared_error(y_test, null_pred, squared = False)

In [13]:
print("Baseline RMSE:", baseline)
print(" ")
print("RMSE:", rmse)
print("________________________________")
print(" ")
print("Difference:", baseline - rmse)

Baseline RMSE: 0.9847035992726725
 
RMSE: 0.03831867954578737
________________________________
 
Difference: 0.9463849197268851


In [14]:
print("RMSE:", rmse)
print(" ")
print("Training Score:", xgb.score(X_train, y_train))
print(" ")
print("Testing Score:", xgb.score(X_test, y_test))

RMSE: 0.03831867954578737
 
Training Score: 0.998554503519591
 
Testing Score: 0.9429373281640024


## GridSearch

In [15]:
xgb = XGBRegressor()

In [16]:
model_params = {}
count = 0

In [19]:
xgb_params = {
    'base_score' : [.25, .30, .35],
    'booster' : ['gbtree', 'gblinear', 'dart'],
    'learning_rate': [.15, .20, .25],
    'max_depth' : [2, 3, 4],
    'n_estimators': [400, 500, 600],
}

gs = GridSearchCV(xgb, param_grid = xgb_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient = 'index')

model_df

,base_score,booster,learning_rate,max_depth,n_estimators,best_score,training_score,testing_score
model_1,0.15,dart,0.15,3,300,0.959702,0.991233,0.958974
model_2,0.25,gbtree,0.20,3,500,0.960058,0.996267,0.960529
model_3,0.25,dart,0.15,3,600,0.961807,0.996040,0.963466
